# <font color = 'green'>**Capstone Project : Segmenting and Clustering neighbourhood in Toronto**<font>

## <font color = 'blue'>Applied Data Science Capstone by IBM/Coursera<font>

### Introduction : This notebook uses the Geocoder package to define the latitude and longitude of the neighborhoods created in Part1 of the project and cluster them on  map

### *Since this is a new notebook created for the latitude and longitude data, <br> the code from the earlier notebook is executed to create our neighbourhood dataframe*

In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
!conda install -c conda-forge geopy --yes # 
#import geocoder # import geocoder
from geopy.geocoders import Nominatim 


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Below code section is the repeat from the first part

In [4]:
#create dataframe fro wikipage
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dtoronto = pd.read_html(url, match='.+', flavor=None, header=None, index_col=None, skiprows=None, attrs=None, parse_dates=False, tupleize_cols=None, thousands=', ', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True)

dfToronto=dtoronto[0]
#create data frame with assigned boroughs
df_assigned=dfToronto[dfToronto.Borough!='Not assigned']
df_assigned.head(7)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned


In [5]:
df_temp = pd.DataFrame(df_assigned,columns=['Postcode'])
dfList = df_temp['Postcode'].unique()
dfPC = pd.DataFrame(dfList,columns=['Postcode'])
dfFinal= pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])
dList = list()
for index, row in dfPC.iterrows():
    pc = row['Postcode']
    nconcat =''
    #print(pc)
    #dfFinal.append(pc)
    for index, irow in df_assigned.iterrows():
        #borough = irow['Borough']
        neigh = irow['Neighbourhood']
        if neigh == 'Not assigned':
            neigh = irow['Borough']
        postcode = irow['Postcode']
        if pc == postcode:
            borough = irow['Borough']
            nconcat = nconcat+ ',' + neigh
    templist = [pc,borough,nconcat]
    dList.append(templist)
# append list to teh final dataframe
dfFinal = pd.DataFrame(dList,columns=['Postcode','Borough','Neighbourhood'])  
print('Dataframe created!')

Dataframe created!


### <font color ='orange'>Below section refers to the process for appending the latitude and longitude of the neighborhood to the dataframe. <br> The csv file containing the coordinates is used instead of the geocoder. This is because repeated attempts from geocoder failed to give coordinates<font>

In [6]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [7]:
#read csv file to the dataframe and check the data
latlon_df = pd.read_csv('Geospatial_Coordinates.csv')
latlon_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
latlon_df['Latitude'].dtype

dtype('float64')

### *Add columns latitude and longitude to the neighbourhood dataframe with type as float*

In [10]:
dfFinal['Latitude']=0.00
dfFinal['Longitude']=0.00
dfFinal['Latitude'].astype(float)
dfFinal['Longitude'].astype(float)

dfFinal.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,",Parkwoods",0.0,0.0
1,M4A,North York,",Victoria Village",0.0,0.0
2,M5A,Downtown Toronto,",Harbourfront,Regent Park",0.0,0.0
3,M6A,North York,",Lawrence Heights,Lawrence Manor",0.0,0.0
4,M7A,Queen's Park,",Queen's Park",0.0,0.0


### *Update column values with the data from the csv file*

In [11]:
latlon = list()

for index, row in latlon_df.iterrows():
    pc = row['Postal Code']
    latitude = row['Latitude']
    longitude = row['Longitude']
    for index, irow in dfFinal.iterrows():
        postcode = irow['Postcode']
        if pc == postcode:
            dfFinal['Latitude'].values[index]=latitude
            dfFinal['Longitude'].values[index]=longitude


In [12]:
dfFinal.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,",Parkwoods",43.753259,-79.329656
1,M4A,North York,",Victoria Village",43.725882,-79.315572
2,M5A,Downtown Toronto,",Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,",Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,",Queen's Park",43.662301,-79.389494


## <font color ='orange'>The below section deals with mapping the neighbourhoods on a map and creating clusters of the neighbourhood on the map<font>

In [13]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


altair-3.2.0         | 770 KB    | #################################

### <font color ='grey'>Map the neighbourhood with folium map. All neighbourhoods are clustered on the map<font>

In [21]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [22]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfFinal['Latitude'], dfFinal['Longitude'], dfFinal['Borough'], dfFinal['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### <font color='grey'> Let us cluster and segment on a selected borough. Let's say we cluster on the borough Downtown Toronto. <br> For this we need to slice our Dataframe to create a dataframe with only Downtown Toronto as the borough<font>

In [30]:
df_downtown = dfFinal[dfFinal['Borough']=='Downtown Toronto'].reset_index(drop=True)
df_downtown.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,",Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,",Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,",St. James Town",43.651494,-79.375418
3,M5E,Downtown Toronto,",Berczy Park",43.644771,-79.373306
4,M5G,Downtown Toronto,",Central Bay Street",43.657952,-79.387383


#### Get the coordinates for the neighbourhood

In [31]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [33]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

## <font color = 'orange'> Explore the neighborhood of Downtown Toronto using foursquare API <font>

In [25]:
CLIENT_ID = '1IFAM5PZF250GJTAHHTBTLP2FDYGUKIRWVMRUXA4NWSNIMNP' 
CLIENT_SECRET = 'YVG5UEPH1FIJYIMTCEC4HL1FE5NGUFOXLNQ3FGIWTPN4JV3V' 
VERSION = '20190919' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1IFAM5PZF250GJTAHHTBTLP2FDYGUKIRWVMRUXA4NWSNIMNP
CLIENT_SECRET:YVG5UEPH1FIJYIMTCEC4HL1FE5NGUFOXLNQ3FGIWTPN4JV3V


In [41]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

### <font color = 'grey'> Let us explore any neighborhood of Downtown Toronto <font>

In [45]:
df_downtown.loc[1, 'Neighbourhood']

neighborhood_latitude = df_downtown.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_downtown.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_downtown.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ,Ryerson,Garden District are 43.6571618, -79.37893709999999.


In [46]:
radius = 500
LIMIT = 25

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1IFAM5PZF250GJTAHHTBTLP2FDYGUKIRWVMRUXA4NWSNIMNP&client_secret=YVG5UEPH1FIJYIMTCEC4HL1FE5NGUFOXLNQ3FGIWTPN4JV3V&v=20190919&ll=43.6571618,-79.37893709999999&radius=500&limit=25'

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d835a64a87921002ce97b4c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Toronto',
  'headerFullLocation': 'Downtown Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 126,
  'suggestedBounds': {'ne': {'lat': 43.6616618045, 'lng': -79.37272880013877},
   'sw': {'lat': 43.652661795499995, 'lng': -79.3851453998612}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57eda381498ebe0e6ef40972',
       'name': 'UNIQLO ユニクロ',
       'location': {'address': '220 Yonge St',
        'crossStreet': 'at Dundas St W',
        'lat': 43.65591027779457,
        'lng': -79.38064099181345,
        'labeledLatLngs': [{'

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
#Now we are ready to clean the json and structure it into a *pandas* dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Blaze Pizza,Pizza Place,43.656518,-79.380015
2,Silver Snail Comics,Comic Shop,43.657031,-79.381403
3,Burrito Boyz,Burrito Place,43.656265,-79.378343
4,Yonge-Dundas Square,Plaza,43.656054,-79.380495
5,Page One Cafe,Café,43.657772,-79.376073
6,DAVIDsTEA,Tea Room,43.656586,-79.381167
7,Ed Mirvish Theatre,Theater,43.655102,-79.379768
8,Hokkaido Ramen Santouka らーめん山頭火,Ramen Restaurant,43.656435,-79.377586
9,Salad King,Thai Restaurant,43.657601,-79.381620


### <font color = 'grey'> Venues returned by Foursquare<font>

In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.
